In [ ]:
# pip install spacy
# python -m spacy download en_core_web_sm
# note: i had to restart my computer after downloading the language package for it to work

In [1]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [3]:
file = pd.read_csv("joined_data_v1.csv")

In [4]:
file = pd.DataFrame(file)
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
1,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
3,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."


In [5]:
file.drop_duplicates(keep='first', inplace=True)

In [6]:
file.head()

,id,imdb_id,title,release_date,spoken_languages,genres1,genres2,genres3,genres4,genres5,...,director,poster_link,production_company,vote_average,vote_average (%),Internet_Movie_Database_rating,Rotten_Tomatoes_rating,Metacritic_rating,rating_average,writer
0,339408,tt4196450,The Birth of a Nation,9/9/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,NaN,NaN,NaN,NaN,...,Nate Parker,https://m.media-amazon.com/images/M/MV5BMTU5Mj...,Fox Searchlight Pictures,6.4,64%,64%,72%,69%,67%,"Nate Parker (screenplay by), Nate Parker (stor..."
2,363676,tt3263904,Sully,9/7/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,History,NaN,NaN,NaN,...,Clint Eastwood,https://m.media-amazon.com/images/M/MV5BY2NmZD...,Warner Bros.,7.0,70%,75%,86%,74%,76%,"Todd Komarnicki, Chesley Sullenberger (based o..."
4,296524,tt1860357,Deepwater Horizon,9/29/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Action,NaN,NaN,NaN,...,Peter Berg,https://m.media-amazon.com/images/M/MV5BOTEzND...,Lionsgate,6.5,65%,71%,83%,68%,72%,"Matthew Michael Carnahan (screenplay by), Matt..."
6,283366,tt1935859,Miss Peregrine's Home for Peculiar Children,9/28/2016,"[{'iso_639_1': 'en', 'name': 'English'}",Drama,Fantasy,Adventure,NaN,NaN,...,Tim Burton,https://m.media-amazon.com/images/M/MV5BMTU0Nz...,20th Century Fox,6.5,65%,67%,65%,57%,64%,Ransom Riggs (based upon the novel written by)...
8,284564,tt3835080,31,9/15/2016,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Horror,Thriller,NaN,NaN,NaN,...,Rob Zombie,https://m.media-amazon.com/images/M/MV5BNTE0NT...,Bow and Arrow Entertainment,4.9,49%,51%,49%,35%,46%,Rob Zombie


In [ ]:
# IGNORE THIS
#creating a list of the overview descriptions we want to run through spacy

# overview_list = file["overview"].tolist()
# overview_list

In [ ]:
# IGNORE THIS
# overview1 = overview_list[0]
# overview1

In [ ]:
# IGNORE THIS
# overview_df = file[["overview"]]
# overview_df

In [7]:
overview_df2 = file[["title", "overview"]].copy()
overview_df2.head()

,title,overview
0,The Birth of a Nation,"Nat Turner, a former slave in America, leads a..."
2,Sully,"On 15 January 2009, the world witnessed the 'M..."
4,Deepwater Horizon,A story set on the offshore drilling rig Deepw...
6,Miss Peregrine's Home for Peculiar Children,A teenager finds himself transported to an isl...
8,31,ROB ZOMBIE'S S P O O K H A U S 31


In [8]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:
# IGNORE THIS
# from pyspark.sql.types import *
# mySchema = StructType([StructField("overview", StringType(), True)])

In [ ]:
# IGNORE THIS
# df = spark.createDataFrame(overview_df,schema=mySchema)

In [9]:
sentenceData = spark.createDataFrame(overview_df2)
sentenceData.show() 

+--------------------+--------------------+
|               title|            overview|
+--------------------+--------------------+
|The Birth of a Na...|Nat Turner, a for...|
|               Sully|On 15 January 200...|
|   Deepwater Horizon|A story set on th...|
|Miss Peregrine's ...|A teenager finds ...|
|                  31|ROB ZOMBIE'S  S P...|
|             Snowden|CIA employee Edwa...|
|         Blair Witch|Students on a cam...|
|The Magnificent S...|Seven gun men in ...|
|              Morgan|A corporate risk-...|
|      The Dark Tower|The last Gunsling...|
|          Nine Lives|A stuffy business...|
|       Don't Breathe|A group of teens ...|
|       Suicide Squad|From DC Comics co...|
|Kubo and the Two ...|Kubo mesmerizes t...|
|             Ben-Hur|A falsely accused...|
|            War Dogs|Based on the true...|
|  Hell or High Water|A divorced dad an...|
|          Anthropoid|In December 1941,...|
|       Pete's Dragon|Pete is a mysteri...|
|            Rock Dog|When a rad

In [10]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

Tokenizer_eace72d1d9fd

In [11]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized.show(truncate=False)

+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                                      |overview               

In [12]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="stopwords")

In [13]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized.show(truncate=False)

+-------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------

# Spacy NLP

In [14]:
# Load English tokenizer, tagger, parser, NER and word vectors

nlp = spacy.load("en_core_web_sm")

In [17]:
#change length to 193 to get all movies
length = 2
i = 0

#empty dictionary
#nested_dict = {}

while i < length: 
    title = str(tokenized.select("title").collect()[0+i])
    clean_title = title[10:]
    text1 =str(tokenized.select("stopwords").collect()[0+i])
    clean_text = text1[16:]
    docs=nlp(clean_text)
    
# populating dictionary attempt 1 (does not work)
    # nested_dict[clean_title] = {"Nouns":[chunk.text for chunk in docs.noun_chunks], "Verbs":[token.lemma_ for token in docs if token.pos_ == "VERB"], "Adverbs": [token.lemma_ for token in docs if token.pos_ == "ADV"], "Adjectives:", [token.lemma_ for token in docs if token.pos_ == "ADJ"]}

# populating dictionary attempt 2 (also does not work)
#     nested_dict[clean_title]['Nouns'] = [chunk.text for chunk in docs.noun_chunks]
#     nested_dict[clean_title]['Verbs'] = [token.lemma_ for token in docs if token.pos_ == "VERB"]
#     nested_dict[clean_title]['Adverbs'] = [token.lemma_ for token in docs if token.pos_ == "ADV"] 
#     nested_dict[clean_title]['Adjectives'] = [token.lemma_ for token in docs if token.pos_ == "ADJ"]
    
#    print(nested_dict)

    print("Movie_Title:", clean_title)
    print("Nouns:", [chunk.text for chunk in docs.noun_chunks])
    print("Verbs:", [token.lemma_ for token in docs if token.pos_ == "VERB"])
    print("Adverbs:", [token.lemma_ for token in docs if token.pos_ == "ADV"])
    print("Adjectives:", [token.lemma_ for token in docs if token.pos_ == "ADJ"])
    
    print("")
    
    i += 1

Movie_Title: 'The Birth of a Nation')
Nouns: ["'former', 'slave', 'america", "'liberation", ", 'movement", ", 'african-americans", 'virgina', ", 'results", "'retaliation"]
Verbs: ['lead']
Adverbs: ['nat']
Adjectives: ['former', 'free', 'african', 'violent']

Movie_Title: 'Sully')
Nouns: ['15\', \'january\', \'2009,\', \'world\', \'witnessed\', "\'miracle', '"hudson', "'captain", 'sullenberger', "'plane", "'waters", "'hudson', 'river", "'lives", "'reputation", 'career']
Verbs: ['witness', 'glide', 'save', 'herald', 'unfold', 'threaten', 'destroy']
Adverbs: ['sully', 'however', 'even', 'sully']
Adjectives: ['january', 'disabled', 'frigid', 'public', 'unprecedented']

